In [ ]:
import sys
sys.path.append('//anaconda/lib/python2.7/site-packages/')
from IPython.display import display
import numpy as np
import pandas as pd
import scipy as sp
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.cross_validation import KFold
from sklearn.cluster import FeatureAgglomeration
from sklearn import svm
import matplotlib
import matplotlib.pyplot as plt
import nltk
from datetime import datetime as dtdm
from sklearn.linear_model import LinearRegression as LinReg
from sklearn.cross_validation import train_test_split
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist, euclidean
from sklearn.cluster import AgglomerativeClustering
import seaborn as sns
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
from textblob import TextBlob
from datetime import datetime as dt


%matplotlib inline

In [2]:
#read in the data and perform some initial data cleaning
df1 = pd.read_csv('xaa')
df2 = pd.read_csv('xab')
df3 = pd.read_csv('xac')

In [3]:
df = pd.concat([df1,df2,df3])

In [8]:
prices = pd.read_csv('prices_final.csv')

In [4]:
df['first_review'] = pd.to_datetime(df['first_review'],infer_datetime_format=True)
df['last_review'] = pd.to_datetime(df['last_review'],infer_datetime_format=True)
df['calendar_last_scraped'] = pd.to_datetime(df['calendar_last_scraped'],infer_datetime_format=True)
df['last_scraped'] = pd.to_datetime(df['last_scraped'],infer_datetime_format=True)
df['host_since'] = pd.to_datetime(df['host_since'],infer_datetime_format=True)

df['first_review'] = pd.to_numeric((df['first_review']-dt(2005,1,1))/np.timedelta64(1, 's'))
df['last_review'] = pd.to_numeric((df['last_review']-dt(2005,1,1))/np.timedelta64(1, 's'))
df['calendar_last_scraped'] = pd.to_numeric((df['calendar_last_scraped']-dt(2005,1,1))/np.timedelta64(1, 's'))
df['last_scraped'] = pd.to_numeric((df['last_scraped']-dt(2005,1,1))/np.timedelta64(1, 's'))
df['host_since'] = pd.to_numeric((df['host_since']-dt(2005,1,1))/np.timedelta64(1, 's'))

df.drop('calendar_updated',axis=1,inplace=True)

In [5]:
df = df.drop(['name',
  'picture_url',
  'host_name',
  'host_picture_url',
  'street','price'
],axis=1)

In [6]:
df_dummies = pd.get_dummies(df)

In [9]:
FA = FeatureAgglomeration(n_clusters=66)
dim_reduced = pd.DataFrame(FA.fit_transform(df_dummies,y=prices.price))

In [10]:
dim_reduced.columns = ['0',         '1',         '2',         '3',         '4',         '5',
               '6',         '7',         '8',         '9',        '10',        '11',
              '12',        '13',        '14',        '15',        '16',        '17',
              '18',        '19',        '20',        '21',        '22',        '23',
              '24',        '25',        '26',        '27',        '28',        '29',
              '30',        '31',        '32',        '33',        '34',        '35',
              '36',        '37',        '38',        '39',        '40',        '41',
              '42',        '43',        '44',        '45',        '46',        '47',
              '48',        '49',        '50',        '51',        '52',        '53',
              '54',        '55',        '56',        '57',        '58',        '59',
              '60',        '61',        '62',        '63',        '64',        '65']

In [11]:
dim_reduced.shape

(90090, 66)

## Sentiment Analysis

We believed that using sentiment analysis of Airbnb reviews would enhance the color of the dataset and improve our accuracy of pricing prediction overall.  In order to perform sentiment analysis, we first had to clean the dataset of reviews for each listing.  This proved to be quite involved because many listings had commas in them which caused the read CSV function to become confused and split reviews inappropriately.  After the intense data cleaning, we used a library called ``text blob`` to find out the polarity score of each review.  We aggregated and found the average of the reviews and assigned those scores to each listing ID, which we then incorporated into our final model.

In [ ]:
#the dataset was a CSV with comments that had multiple commas
#the CSV wouldn't read properly if you included certain lines
#those lines are dropped below
reviews1 = pd.read_csv('reviewsaa',quoting=csv.QUOTE_NONE,error_bad_lines=False)
reviews2 = pd.read_csv('reviewsab',quoting=csv.QUOTE_NONE,error_bad_lines=False)
reviews3 = pd.read_csv('reviewsac',quoting=csv.QUOTE_NONE,error_bad_lines=False)
reviews4 = pd.read_csv('reviewsad',quoting=csv.QUOTE_NONE,error_bad_lines=False)
reviews5 = pd.read_csv('reviewsae',quoting=csv.QUOTE_NONE,error_bad_lines=False)
reviews6 = pd.read_csv('reviewsaf',quoting=csv.QUOTE_NONE,error_bad_lines=False)
reviews7 = pd.read_csv('reviewsag',quoting=csv.QUOTE_NONE,error_bad_lines=False)

In [12]:
reviews = pd.concat([reviews1,reviews2,reviews3,reviews4,reviews5,reviews6,reviews7])

In [13]:
display(reviews1[:25].T)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Unnamed: 0,1180670,722394,4074444,68046,"Un excellent séjour """,68774,920951,462454,2207015,1773664,...,3595080,571379,571379,571379,4417283,2318157,2318157,It is within walking distance to 2 J,Although there was some initial misunderstandi...,The apartment is on the 3rd floor which was a ...
listing_id,14705995,9248781,23983183,11797670,NaN,23987149,16985911,13159495,13088178,11945863,...,19304641,23484587,22953715,22197128,23387541,24527364,23963288,Z & M lines,Joseph was very responsive and accommodating ...,"so guests just have to be prepared for that."""
id,2014-06-24,2013-12-16,2014-12-15,2014-04-15,NaN,2014-12-15,2014-08-05,2014-05-20,2014-05-19,2014-04-20,...,2014-09-09,2014-12-01,2014-11-18,2014-11-01,2014-11-30,2014-12-30,2014-12-15,to shops and places to eat.,NaN,NaN
date,10875598,6821360,8822691,12231047,NaN,23170531,16614692,14129237,14336410,9979725,...,9812457,22551403,22628020,3884398,23381957,22618355,1663547,NaN,NaN,NaN
reviewer_id,Gregory,Giri,Wendy,Virginie,NaN,Anthony,Anthony,Amanda,Laurie,Eric,...,Matilda,Sudheendra,Zuzka,Alex,Ignacio,Mally,Abigail,NaN,NaN,NaN
reviewer_name,"""Ok",Extremely disappointed.,Exactly as described.,"""Appartement très sympa",NaN,The reservation was canceled the day before ar...,Very Nice and comfortable. Great location - Ma...,Sunserae was a phenomenal host. Incredible loc...,"""Our rental was great. It was just as described","""fantastique",...,Zak was an amazing and flexible host. The plac...,Phillip arranged everything even if he was not...,Philip is a great host. We felt welcomed and h...,Philip was great host! That was nice to meet w...,"""Todo muy bien. El piso muy limpio y el barrio...",Nice location and close to subway. The apartme...,"""Joseph's property was just as pictured. It i...",NaN,NaN,NaN
comments,if you like the location and don't mind an oc...,NaN,NaN,accueillant. A quelques pas du métro. Tout ét...,NaN,NaN,NaN,NaN,clean and the perfect location. I highly reco...,NaN,...,NaN,NaN,NaN,NaN,"más seguro de lo que pensaba.""",NaN,peaceful street which was perfect for us for ...,NaN,NaN,NaN


As you can see above, the commas in the comments confused the CSV reader and caused the data to be read in incorrectly.  You can see this in the rows in which ``reviewer_name`` and ``comments`` have split up strings and the names of the reviewers have been shifted up to ``reviewer_id.``  The next kernels will work to ameliorate this issue, as well as fix the NaNs in the dataset.

In [14]:
reviews['comments_combined'] = ''

reviews = reviews[pd.notnull(reviews.listing_id)&pd.notnull(reviews.id)&pd.notnull(reviews.date)]

In [13]:
display(reviews[:25].T)

,0,1,2,3,5,6,7,8,9,11,...,17,18,19,20,21,25,26,27,28,29
Unnamed: 0,1180670,722394,4074444,68046,68774,920951,462454,2207015,1773664,3414084,...,571379,571379,4417283,2318157,2318157,2318157,2318157,2318157,2318157,2318157
listing_id,14705995,9248781,23983183,11797670,23987149,16985911,13159495,13088178,11945863,21818387,...,22953715,22197128,23387541,24527364,23963288,23875934,23416177,23333842,23133922,22600975
id,2014-06-24,2013-12-16,2014-12-15,2014-04-15,2014-12-15,2014-08-05,2014-05-20,2014-05-19,2014-04-20,2014-10-24,...,2014-11-18,2014-11-01,2014-11-30,2014-12-30,2014-12-15,2014-12-13,2014-11-30,2014-11-29,2014-11-23,2014-11-10
date,10875598,6821360,8822691,12231047,23170531,16614692,14129237,14336410,9979725,240921,...,22628020,3884398,23381957,22618355,1663547,22239307,64455,21393429,23674312,22263724
reviewer_id,Gregory,Giri,Wendy,Virginie,Anthony,Anthony,Amanda,Laurie,Eric,Nancy,...,Zuzka,Alex,Ignacio,Mally,Abigail,Rossina,Sari,Muhammad Zamir,Drew,Kathleen
reviewer_name,"""Ok",Extremely disappointed.,Exactly as described.,"""Appartement très sympa",The reservation was canceled the day before ar...,Very Nice and comfortable. Great location - Ma...,Sunserae was a phenomenal host. Incredible loc...,"""Our rental was great. It was just as described","""fantastique",The reservation was canceled 14 days before ar...,...,Philip is a great host. We felt welcomed and h...,Philip was great host! That was nice to meet w...,"""Todo muy bien. El piso muy limpio y el barrio...",Nice location and close to subway. The apartme...,"""Joseph's property was just as pictured. It i...","""La casa de Joseph está bien para pasar unos días",Close to the J and lots of space!,It is a wonderful stay.. The apartment is spac...,The place was clean and just as expected. Jose...,Joseph made key pick up easy even though we we...
comments,if you like the location and don't mind an oc...,NaN,NaN,accueillant. A quelques pas du métro. Tout ét...,NaN,NaN,NaN,clean and the perfect location. I highly reco...,NaN,NaN,...,NaN,NaN,"más seguro de lo que pensaba.""",NaN,peaceful street which was perfect for us for ...,el barrio es tranquilo. No nos gustó mucho qu...,NaN,NaN,NaN,NaN
comments_combined,,,,,,,,,,,...,,,,,,,,,,


In [15]:
nan_reviews = reviews[pd.isnull(reviews.comments)]

In [16]:
nan_reviews.comments_combined = nan_reviews.reviewer_name
nan_reviews.drop(['reviewer_name','comments'],axis=1,inplace=True)
display(nan_reviews.T)

//anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,1,2,5,6,7,9,11,12,14,15,...,18302,18303,18304,18305,18308,18309,18314,18317,18325,18328
Unnamed: 0,722394,4074444,68774,920951,462454,1773664,3414084,4594212,3595080,3595080,...,2097119,2097119,2097119,2097119,2097119,2097119,2097119,2097119,2097119,Would stay here again
listing_id,9248781,23983183,23987149,16985911,13159495,11945863,21818387,24494076,21605402,19304641,...,15894148,15778887,15507528,15464274,12246920,11813927,11007129,10431089,10275691,and again
id,2013-12-16,2014-12-15,2014-12-15,2014-08-05,2014-05-20,2014-04-20,2014-10-24,2014-12-29,2014-10-20,2014-09-09,...,2014-07-17,2014-07-15,2014-07-10,2014-07-09,2014-04-27,2014-04-15,2014-03-17,2014-02-18,2014-02-10,and again
date,6821360,8822691,23170531,16614692,14129237,9979725,240921,24667083,21614832,9812457,...,17977720,17151273,17240467,2046899,11835834,2973994,11531257,7492800,5873351,but alas I am apartment hunting
reviewer_id,Giri,Wendy,Anthony,Anthony,Amanda,Eric,Nancy,Tai-Hui,Xuran,Matilda,...,Lily,Heather,Cindy,Darcus,Leonie,Ankit,Raffaela,Claire,Triphina,so hoping you will enjoy as much as I did. O...
comments_combined,Extremely disappointed.,Exactly as described.,The reservation was canceled the day before ar...,Very Nice and comfortable. Great location - Ma...,Sunserae was a phenomenal host. Incredible loc...,"""fantastique",The reservation was canceled 14 days before ar...,It's a very convenient place and the host is v...,这个房间很棒，社区很安静，房东也非常热情，阳光也很好。,Zak was an amazing and flexible host. The plac...,...,Ory is so nice !He just gives me a lot of surp...,Had a brilliant stay at Ory's. Great location ...,I took my daughter to NYC for an overnight tri...,"""Ory and his sister Dani were both very hospit...",Orys Location in NYC is really well located in...,Ory was a great host and the place was really ...,Ory was a great Host! Thank your for the good ...,"""I had a fantastic stay at Ory's apartment. Ev...",Ory has a lovely apartment in a great location...,"2014"""


In [17]:
is_reviews = reviews[pd.notnull(reviews.comments)]
is_reviews.comments_combined = is_reviews.reviewer_name+is_reviews.comments
is_reviews.drop(['reviewer_name','comments'],axis=1,inplace=True)
display(is_reviews.T)

//anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,0,3,8,13,19,21,25,32,35,57,...,18283,18285,18287,18290,18293,18296,18301,18307,18310,18330
Unnamed: 0,1180670,68046,2207015,3595080,4417283,2318157,2318157,3875386,2893165,1210004,...,2097119,2097119,2097119,2097119,2097119,2097119,2097119,2097119,2097119,4194302
listing_id,14705995,11797670,13088178,22861687,23387541,23963288,23875934,18504345,13011902,23743047,...,22097556,20695464,19919250,19392382,19025459,17954629,16475764,13010471,11484543,23102218
id,2014-06-24,2014-04-15,2014-05-19,2014-11-16,2014-11-30,2014-12-15,2014-12-13,2014-08-27,2014-05-17,2014-12-09,...,2014-10-30,2014-10-04,2014-09-21,2014-09-11,2014-09-05,2014-08-19,2014-07-28,2014-05-17,2014-04-05,2014-11-23
date,10875598,12231047,14336410,23388111,23381957,1663547,22239307,20263851,14781240,12472288,...,13753713,14871313,11845462,20950556,17062887,17522090,16806979,14265884,8727209,22417130
reviewer_id,Gregory,Virginie,Laurie,Mark,Ignacio,Abigail,Rossina,Thomas,Yossi,Ida,...,Federico,Christian,Kevin,Wouter,Cheryl,Daphné,Marco,Olga,Joana,Dante
comments_combined,"""Ok if you like the location and don't mind an...","""Appartement très sympa accueillant. A quelque...","""Our rental was great. It was just as describe...","""Zak and his room were most hospital in receiv...","""Todo muy bien. El piso muy limpio y el barrio...","""Joseph's property was just as pictured. It i...","""La casa de Joseph está bien para pasar unos d...","""I felt very welcome and the apartment was gre...","""Great apartment awesome location. Recommended...","""Amazing location quiet & well-equiped. We lov...",...,"""La experiencia fue muy buena desde todo punto...","""Great Host very friendly. Perfect location fo...","""Ory is a great host makes you feel welcome an...","""O was welcoming friendly and available. Pleas...","""What a great location close to all the popula...","""We had a great time in NYC the apartment is l...","""This is a really great place and Ory is a ver...","""We had a very short stay at Ory's place. Howe...","""Le séjour s'est super bien passé. L’apparteme...","""We had a superb time staying at Daniel and La..."


In [18]:
reviews = pd.concat([nan_reviews,is_reviews])
reviews.columns = ['listing_id','id','date','reviewer_id','reviewer_name','comments']
reviews = reviews[pd.notnull(reviews.comments)]
display(reviews[0:50].T)

,1,2,5,6,7,9,11,12,14,15,...,66,70,71,72,79,88,92,100,101,102
listing_id,722394,4074444,68774,920951,462454,1773664,3414084,4594212,3595080,3595080,...,3239266,3239266,999248,999248,999248,2067161,As soon as we arrived,3110734,3176792,3176792
id,9248781,23983183,23987149,16985911,13159495,11945863,21818387,24494076,21605402,19304641,...,24294620,23833621,24219267,24100979,23030059,17734206,we were hungry,13265778,19918553,19680602
date,2013-12-16,2014-12-15,2014-12-15,2014-08-05,2014-05-20,2014-04-20,2014-10-24,2014-12-29,2014-10-20,2014-09-09,...,2014-12-25,2014-12-12,2014-12-23,2014-12-20,2014-11-21,2014-08-16,Isabelle helped us prepare our meal,2014-05-23,2014-09-20,2014-09-16
reviewer_id,6821360,8822691,23170531,16614692,14129237,9979725,240921,24667083,21614832,9812457,...,4392785,19493140,7183106,20821287,16803151,17602217,"it was very nice. She advised us to do some ""...",15832505,21058993,19086701
reviewer_name,Giri,Wendy,Anthony,Anthony,Amanda,Eric,Nancy,Tai-Hui,Xuran,Matilda,...,Miaoxin,Tiago,Cami,Stephanie,Alie,Steven,moma or MET,Graciela,Nicole,Kirstin
comments,Extremely disappointed.,Exactly as described.,The reservation was canceled the day before ar...,Very Nice and comfortable. Great location - Ma...,Sunserae was a phenomenal host. Incredible loc...,"""fantastique",The reservation was canceled 14 days before ar...,It's a very convenient place and the host is v...,这个房间很棒，社区很安静，房东也非常热情，阳光也很好。,Zak was an amazing and flexible host. The plac...,...,It was a great stay. Perfect location. Only fi...,Excellent Location and comfortable beds! Layka...,It was awesome. Jon was very helpful!,We loved the apartment. Fabulous space and gre...,Wij hebben genoten van de loft. Heerlijk om na...,The reservation was canceled 65 days before ar...,do the memorial day for example.,The reservation was canceled 6 days before arr...,The reservation was canceled the day before ar...,Jamba is truly a very easy going landlord. It ...


The data cleaning of the reviews is finally complete.  It is now time to apply sentiment analysis to the cleaned dataset.

In [19]:
#apply the sentiment analysis to each review
tok_reviews = reviews.comments.apply(lambda row: TextBlob(row).sentiment)

In [20]:
reviews['tok_reviews'] = tok_reviews

In [24]:
reviews.shape

(155052, 9)

In [21]:
#create a series of the polarity and subjectivity score of each review
reviews[['polarity', 'subjectivity']] = reviews['tok_reviews'].apply(pd.Series)

In [22]:
display(reviews[:25].T)

,1,2,5,6,7,9,11,12,14,15,...,27,28,29,30,31,33,34,36,37,38
listing_id,722394,4074444,68774,920951,462454,1773664,3414084,4594212,3595080,3595080,...,2318157,2318157,2318157,2318157,3163306,2975505,2975505,3529799,3529799,11900
id,9248781,23983183,23987149,16985911,13159495,11945863,21818387,24494076,21605402,19304641,...,23333842,23133922,22600975,21696459,20824314,20048857,19418134,17734158,17734156,24328669
date,2013-12-16,2014-12-15,2014-12-15,2014-08-05,2014-05-20,2014-04-20,2014-10-24,2014-12-29,2014-10-20,2014-09-09,...,2014-11-29,2014-11-23,2014-11-10,2014-10-21,2014-10-06,2014-09-22,2014-09-12,2014-08-16,2014-08-16,2014-12-26
reviewer_id,6821360,8822691,23170531,16614692,14129237,9979725,240921,24667083,21614832,9812457,...,21393429,23674312,22263724,20401323,18996419,21075905,19505327,18167483,18167483,23384901
reviewer_name,Giri,Wendy,Anthony,Anthony,Amanda,Eric,Nancy,Tai-Hui,Xuran,Matilda,...,Muhammad Zamir,Drew,Kathleen,Stefan,Julieta,Rachel,Hugo,Anne,Anne,Sven Ole
comments,Extremely disappointed.,Exactly as described.,The reservation was canceled the day before ar...,Very Nice and comfortable. Great location - Ma...,Sunserae was a phenomenal host. Incredible loc...,"""fantastique",The reservation was canceled 14 days before ar...,It's a very convenient place and the host is v...,这个房间很棒，社区很安静，房东也非常热情，阳光也很好。,Zak was an amazing and flexible host. The plac...,...,It is a wonderful stay.. The apartment is spac...,The place was clean and just as expected. Jose...,Joseph made key pick up easy even though we we...,The reservation was canceled the day before ar...,El departamento y la ubicación son excelentes....,This was a great place! Two blocks from the su...,We had a great time staying with Nate. The typ...,The reservation was canceled 36 days before ar...,The reservation was canceled 32 days before ar...,Zuverlässig und kooperativ.
tok_reviews,"(-0.75, 0.75)","(0.25, 0.25)","(0.0, 0.0)","(0.5950000000000001, 0.8374999999999999)","(0.6025, 0.735)","(0.0, 0.0)","(0.0, 0.0)","(0.66, 0.6833333333333332)","(0.0, 0.0)","(0.20454545454545459, 0.5765151515151515)",...,"(0.43533333333333335, 0.8019999999999999)","(0.26666666666666666, 0.6)","(0.15833333333333333, 0.5363756613756614)","(0.0, 0.0)","(0.0, 0.0)","(0.8055555555555555, 0.8055555555555555)","(0.5166666666666667, 0.7699999999999999)","(0.0, 0.0)","(0.0, 0.0)","(0.0, 0.0)"
polarity,-0.75,0.25,0,0.595,0.6025,0,0,0.66,0,0.204545,...,0.435333,0.266667,0.158333,0,0,0.805556,0.516667,0,0,0
subjectivity,0.75,0.25,0,0.8375,0.735,0,0,0.683333,0,0.576515,...,0.802,0.6,0.536376,0,0,0.805556,0.77,0,0,0


In [23]:
reviews.to_csv('sentiment_reviews.csv')

In [21]:
groups = reviews[reviews.listing_id.apply(lambda x: x.isnumeric())]

In [22]:
#find the average polarity score of each listing and assign it to each listing ID
groups = groups.groupby(['listing_id'])['polarity'].mean()

In [45]:
groups

listing_id
100002     0.324434
1000083    0.166426
100023     0.236042
1000264    0.474763
1000310    0.321938
1000959    0.359767
1001145    0.212213
1001147    0.358349
1001353    0.427501
1001405    0.250000
1001447    0.447807
1001648    0.311905
100184     0.218624
100186     0.336251
1002469    0.434038
1002645    0.345708
1002714    0.464857
1003530    0.333280
1003585    0.372837
1003634    0.477723
100368     0.430119
1003754    0.427134
1003840    0.393997
1003930    0.668182
1004007    0.512500
1004162    0.395457
1004291    0.367076
1004427    0.131349
1005416    0.465926
1005558    0.444597
             ...   
992633     0.000000
992891     0.379848
992896     0.451259
993011     0.537143
993040     0.378008
993785     0.394371
994638     0.366269
995187     0.532024
995341     0.242250
995367     0.424653
995373     0.397217
995438     0.457040
995926     0.677778
996245     0.491888
996607     0.362867
996625     0.379816
996790     0.218343
997028     0.297809
997173   

In [34]:
df_dummies.shape

(90090, 356)

In [47]:
display(groups[:5].T)

listing_id
100002     0.324434
1000083    0.166426
100023     0.236042
1000264    0.474763
1000310    0.321938
Name: polarity, dtype: float64

In [48]:
print(groups.index.dtype)

object


In [151]:
print(new_unique[:5])

[722394, 4074444, 68774, 920951, 462454]


In [23]:
#show each listing ID and its polarity score
groups = pd.DataFrame(groups)
int_groups_idx = pd.to_numeric(groups.index)
display(groups)

,polarity
listing_id,
100002,0.324434
1000083,0.166426
100023,0.236042
1000264,0.474763
1000310,0.321938
1000959,0.359767
1001145,0.212213
1001147,0.358349
1001353,0.427501


In [24]:
groups['idx'] = int_groups_idx

In [25]:
groups.set_index('idx',inplace=True)

In [55]:
display(groups)

,polarity
idx,
100002,0.324434
1000083,0.166426
100023,0.236042
1000264,0.474763
1000310,0.321938
1000959,0.359767
1001145,0.212213
1001147,0.358349
1001353,0.427501


In [37]:
df_dummies.set_index('id',inplace=True)

In [26]:
df_dummies['sentiment'] = 0.0

In [ ]:
#assign polarity scores to entire reviews dataset
n = 0
for i in groups.index:

    df_dummies.loc[i,'sentiment'] = groups.loc[i].polarity
    n += 1
    if (n % 100) == 0:
            print(n)

In [49]:
#write the final results to a csv
df_dummies.to_csv('time_sentiment.csv')